### Backstory
Given a specific user, find out more about them: where they’re from, what things they like/dislike, and other data about their background (think of at least 2 more things to determine). Note that this should be somewhat automated; I should be able to give you a username and you’ll produce a backstory dataset for them (it may work better with some users than others). Provide a three sample user backstories in your report, written up as a narrative.

In [73]:
import pandas as pd
from pyspark.sql.types import StructType, StructField, FloatType, LongType, StringType, IntegerType
from  pyspark.sql.functions import input_file_name
from pyspark.sql import SQLContext, Row


df = sqlContext.read.json("hdfs://orion11:32001/sampled_reddit/*")


columns = [
    "author", "author_cakeday", "name", "body", "author_flair_text", "gilded", "score_hidden", "score", "link_id",
        "retrieved_on", "author_flair_css_class", "subreddit", "edited", "ups", "downs", "controversiality",
        "created_utc", "parent_id", "archived", "subreddit_id", "id", "distinguished"]

df.select("author").show(n=50)

+------------------+
|            author|
+------------------+
|        halftone84|
|         keiyakins|
|    chuck_finley17|
|            iceage|
|   phenomenonymous|
|  tumbleweed1993sf|
|           zero_iq|
|   gabrielbenjamin|
|           Ishkabo|
|          ihrtgngr|
|         [deleted]|
|         Gareth321|
|           BigDisk|
|        robdidlyob|
|         danxmason|
|            LeBrad|
|          DoubleKK|
|         [deleted]|
|          Ebeforei|
|     ultimatechase|
|             pilaf|
|             Lazek|
|       jaggederest|
|         [deleted]|
|       RobotBuddha|
|urmomlikesarrested|
|         [deleted]|
|       DeadlyTedly|
|              4ray|
|         naaahhman|
|   frequencypromos|
|           GotPerl|
|           Atheizt|
|             eepba|
|     TimesAreTough|
|         [deleted]|
|         bacontaco|
|      moarroidsplz|
|        FrankCraft|
|         SamAllmon|
|          briesa37|
|         Chargelot|
|         [deleted]|
|         spastacus|
|          gl

In [95]:
#get all comments user left in subreddits, user's flair_text, flair_css_class and controversiality
user_name = 'jaggederest'
pd_comments = spark.sql("""select temp_df.created_utc, temp_df.subreddit, temp_df.body, temp_df.author_flair_text,
temp_df.author_flair_css_class, temp_df.controversiality
from TEMP_DF 
where temp_df.author = '""" + user_name + "'").toPandas()
pd_comments

,created_utc,subreddit,body,author_flair_text,author_flair_css_class,controversiality
0,1309478485,tf2,"Also parts of Louisiana, Maine, Vermont, New H...",None,None,0
1,1309667560,Frugal,I work a primary job and a spare-time-as-a-hob...,None,None,0
2,1313976011,wikipedia,"You know, we can find out, through SCIENCE.\n\...",None,None,0
3,1314005864,wikipedia,"Scooters are neat, bro.",None,None,0
4,1249085615,reddit.com,"You got it, buddy, that's the one. As bold and...",None,None,0
5,1249161084,programming,"Oh, man, now I need to go find that on youtube...",None,None,0
6,1249254952,philosophy,The dictionary uses unrefined versions of the ...,None,None,0
7,1249258078,technology,&gt; yet China is just as market driven as the...,None,None,0
8,1249258346,bestof,One cannot simply fuck into /r/meetup,None,None,0
9,1249260077,philosophy,Maybe I was miseducated. My philosophy teacher...,None,None,0


In [96]:
pd_comments.to_csv('backstory_'+user_name+'.csv')